# Running and Managing Multiple Experiments with SageMaker Experiments

<img align="left" width="130" src="https://raw.githubusercontent.com/PacktPublishing/Amazon-SageMaker-Cookbook/master/Extra/cover-small-padded.png"/>

This notebook contains the code to help readers work through one of the recipes of the book [Machine Learning with Amazon SageMaker Cookbook: 80 proven recipes for data scientists and developers to perform ML experiments and deployments](https://www.amazon.com/Machine-Learning-Amazon-SageMaker-Cookbook/dp/1800567030)

### How to do it...

In [ ]:
import sagemaker, boto3

session = boto3.Session()
sagemaker_session = sagemaker.Session(boto_session=session)
sagemaker_client = session.client('sagemaker')
role = sagemaker.get_execution_role()

In [ ]:
import time, os, sys
import numpy as np
import pandas as pd
import itertools

from pprint import pprint

In [ ]:
!pip install sagemaker-experiments

In [ ]:
from smexperiments.experiment import Experiment
from smexperiments.trial import Trial
from smexperiments.trial_component import TrialComponent
from smexperiments.tracker import Tracker

In [ ]:
import random
import string

def generate_random_string():
    list_of_chars = random.choices(
        string.ascii_uppercase, 
        k=10)
    return ''.join(list_of_chars)

In [ ]:
label = generate_random_string() 
training_experiment = Experiment.create(
    experiment_name = f"experiment-{label}",
    description     = "Experiment Description",
    sagemaker_boto_client=sagemaker_client)

In [ ]:
hyperparam_options = {
    'max_depth': [2, 8],
    'eta': [0.2],
    'gamma': [3, 4],
    'min_child_weight': [6],
    'subsample': [0.4],
    'num_round': [10, 20],
    'objective': ['binary:logistic']
}

In [ ]:
def prepare_hyperparam_variations(options):
    names, values = zip(*options.items())
    return [dict(zip(names, value)) 
            for value in itertools.product(*values)]

In [ ]:
hyperparam_variations = prepare_hyperparam_variations(
    hyperparam_options
)

hyperparam_variations

In [ ]:
s3_bucket = 'sagemaker-cookbook-bucket'
prefix = "chapter05"

s3_bucket = "sagemaker-cookbook-bucket"
prefix = "chapter05"
path = f"s3://{s3_bucket}/{prefix}/input"
training_path = f"{path}/training_data.csv" 
validation_path = f"{path}/validation_data.csv" 
output_path = f"s3://{s3_bucket}/{prefix}/output/"

In [ ]:
from sagemaker.image_uris import retrieve
container = retrieve('xgboost', 
                     boto3.Session().region_name, 
                     version="0.90-2")
container

In [ ]:
from sagemaker.inputs import TrainingInput
    
s3_input_training = TrainingInput(training_path, 
                                  content_type="text/csv")

s3_input_validation = TrainingInput(validation_path, 
                                    content_type="text/csv")

In [ ]:
with Tracker.create(
    display_name="xgboost-experiment-display-name", 
    artifact_bucket=s3_bucket,
    artifact_prefix=training_experiment.experiment_name,
    sagemaker_boto_client=sagemaker_client
) as experiment_tracker:    
    experiment_tracker.log_input(name="training-input", 
                                 media_type="s3/uri", 
                                 value=training_path)
    
    experiment_tracker.log_input(name="validation-input", 
                                 media_type="s3/uri", 
                                 value=validation_path)
    
    experiment_tracker.log_parameters(hyperparam_options)

In [ ]:
experiment_tracker.__dict__

In [ ]:
def track_and_generate_config(
    experiment_tracker, 
    experiment_name, 
    job_name, 
    random_string, 
    hyperparameters):
    
    tracker_display_name = f"trial-metadata-{random_string}"
    print(f"{label} Create Tracker: {tracker_display_name}")
    
    with Tracker.create(
        display_name=tracker_display_name,
        artifact_bucket="sagemaker-cookbook-bucket",
        artifact_prefix=f"{experiment_name}/{job_name}",
        sagemaker_boto_client=sagemaker_client
    ) as trial_tracker:
        
        trial_tracker.log_parameters(hyperparameters)

    trial_name = f'trial-{random_string}'
    print(f"Create Trial: {trial_name}")
    
    trial = Trial.create(
        trial_name=trial_name, 
        experiment_name=experiment_name,
        sagemaker_boto_client=sagemaker_client)
    
    trial.add_trial_component(
        experiment_tracker.trial_component)
    time.sleep(1) 
    trial.add_trial_component(
        trial_tracker.trial_component)
    
    print(f"Prepare Experiment Configuration")

    return {
        "ExperimentName": experiment_name, 
        "TrialName": trial.trial_name,
        "TrialComponentDisplayName": job_name
    }

In [ ]:
import time

experiment_name = training_experiment.experiment_name

for index, hyperparameters in enumerate(
    hyperparam_variations
):
    iteration = index + 1
    print(f"Iteration # {iteration}")
    label = f"[Iteration # {iteration}]"
    random_string = generate_random_string()
    job_name = f"job-{random_string}"
    
    print(f"{label} Track and Generate Config")
    experiment_config = track_and_generate_config(
        experiment_tracker=experiment_tracker,
        experiment_name=experiment_name,
        job_name=job_name,
        random_string=random_string,
        hyperparameters=hyperparameters)
    
    time.sleep(1)
    print(f"{label} Initialize Estimator")
    estimator = sagemaker.estimator.Estimator( 
        container,
        role,
        instance_count=1, 
        instance_type='ml.m5.large', 
        output_path=output_path, 
        hyperparameters=hyperparameters,
        enable_sagemaker_metrics = True,
        sagemaker_session=sagemaker_session
    )
    
    print(f"{label} Call fit() function")
    estimator.fit({'train': s3_input_training, 
                   'validation': s3_input_validation},
                   job_name = job_name, 
                   wait=False,
                   experiment_config=experiment_config)

In [ ]:
experiment_name = training_experiment.experiment_name
experiment_name

In [ ]:
%store experiment_name